In [ ]:
# Import local modules
import sys
import os
import glob

# Import third-party modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import iris
from xarray import DataArray

In [ ]:
import dictionaries as dicts

from functions import *

In [ ]:
# First, we want to visualise th gridboxes which we might want to use
# The UK grid box or the north sea grid box

# Set up the projection
projection = ccrs.PlateCarree(central_longitude=0)

# Plot the differences between the two gridboxes
# Set up the figure
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=projection)

# # Include the stock image and coastlines
ax.stock_img()
ax.coastlines()

# Extract the lat and lon data for the UK box
lon1_uk, lon2_uk = dicts.uk_grid['lon1'], dicts.uk_grid['lon2']
lat1_uk, lat2_uk = dicts.uk_grid['lat1'], dicts.uk_grid['lat2']

# Extract the lat and lon data for the North Sea box
lon1_ns, lon2_ns = dicts.north_sea_kay['lon1'], dicts.north_sea_kay['lon2']
lat1_ns, lat2_ns = dicts.north_sea_kay['lat1'], dicts.north_sea_kay['lat2']

# Plot these boxes
ax.plot([lon1_uk, lon2_uk, lon2_uk, lon1_uk, lon1_uk],
        [lat1_uk, lat1_uk, lat2_uk, lat2_uk, lat1_uk],
        color='green', linewidth=2, transform=projection, label="UK Clark")

# Plot these boxes
ax.plot([lon1_ns, lon2_ns, lon2_ns, lon1_ns, lon1_ns],
        [lat1_ns, lat1_ns, lat2_ns, lat2_ns, lat1_ns],
        color='orange', linewidth=2, transform=projection, label="North Sea Kay")

# Zoom in on the region
ax.set_extent([-50, 20, 40, 80], crs=projection)

# Create a legend
ax.legend(loc = 'upper left')

# show the plot
plt.show()

### UNSEEN temp and wind speeds ###

To get an understanding of the demand and wind speed influences, we want to plot the occurence of yearly cold spells alongside low wind spells.

We are going to make the assumption that for the wind speeds, we care primarily about the North Sea, while for the temperature, we care about the UK box.

### Looking at winds ###

First we want to extract the winds for the North Sea gridbox and perform the fidelity testing.

In [ ]:
# Set up the parameters
model_var = "sfcWind"
obs_var = "si10"
test_model = "HadGEM3-GC31-MM"
experiment = "dcppA-hindcast"

In [ ]:
import importlib

importlib.reload(sys.modules['functions'])

from functions import *

In [ ]:
# Load the model data for the north sea region
model_wind_ns = load_model_data(model_variable=model_var,
                                model=test_model,
                                experiment=experiment,
                                start_year=1960,
                                end_year=2018,
                                avg_period=1,
                                grid=dicts.north_sea_kay)

In [ ]:
# Perform the bootstrapping
# Average over the year
model_stats_ns = model_stats_bs(model=model_wind_ns.mean(axis=2),
                                nboot=10000,)

In [ ]:
# Load the obs data
obs_data_ns = load_obs_data(obs_variable=obs_var,
                            regrid_obs_path="/home/users/benhutch/ERA5/surface_wind_ERA5_regrid_HadGEM.nc",
                            start_year=1960,
                            end_year=2018,
                            avg_period=1,
                            grid=dicts.north_sea_kay)

In [ ]:
print(obs_data_ns[0].shape)

In [ ]:
# Calculate the obs stats
obs_stats_ns = calculate_obs_stats(obs_data=obs_data_ns[0],
                                   start_year=1960,
                                   end_year=2018,
                                   avg_period=1,
                                   grid=dicts.north_sea_kay)

In [ ]:
# Plot the moments of the distribution
plot_moments(model_stats=model_stats_ns,
             obs_stats=obs_stats_ns
             )

1-year RM for the North Sea region appear to be just inside the 5%-95% range of the bootstrapped model data.

It also appears that the mean will need bias adjustment - the models are around ~ 0.4 m/s higher than the obs.

In [ ]:
# Plot the distribution
plot_distribution(model_data=model_wind_ns.mean(axis=2),
                  obs_data=obs_data_ns[0],
                  )

### Fidelity tests ###

Fidelity tests for winds in the North Sea look good (i.e. obs values are within the 5%-95% range of the model data). The mean will need bias adjustment.

Now we can plot the individual events.

In [ ]:
import importlib

importlib.reload(sys.modules['functions'])

from functions import *

In [ ]:
# Plot the year events
# For wind speeds in the north sea region
plot_events(model_data=model_wind_ns.mean(axis=2),
            obs_data=obs_data_ns[0],
            obs_stats=obs_stats_ns,
            start_year=1960,
            end_year=2018,
            figsize_y=5,)

### Temperature data ###

Now we want to find the temperature data, both for the observations (from ERA5) and the model data (probably HadGEM3 again).

In [ ]:
# Set up the parameters
model_var = "tas"
obs_var = "t2m"
test_model = "HadGEM3-GC31-MM"
experiment = "dcppA-hindcast"
obs_path = "/gws/nopw/j04/canari/users/benhutch/ERA5/t2m_ERA5_regrid_HadGEM.nc"

In [ ]:
# Check if the CSV file is empty
if os.path.getsize("/home/users/benhutch/multi_year_unseen/paths/paths_20240117T122513.csv") > 0:
    # Open the csv file in ./paths
    paths = pd.read_csv("/home/users/benhutch/multi_year_unseen/paths/paths_20240117T122513.csv")
else:
    # Handle the empty file case
    paths = pd.DataFrame()

paths

# Add a new row for
# tas, dcppA-hindcast, HadGEM3-GC31-MM, /gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/data/tas/
new_row = {'variable': 'tas',
           'experiment': 'dcppA-hindcast',
              'model': 'HadGEM3-GC31-MM',
                'path': '/gws/nopw/j04/canari/users/benhutch/dcppA-hindcast/data/tas/HadGEM3-GC31-MM/'}

# Append the new row to the dataframe
paths = paths.append(new_row, ignore_index=True)

paths

# Save the dataframe
paths.to_csv("/home/users/benhutch/multi_year_unseen/paths/paths_20240117T122513.csv", index=False)

In [ ]:
# Load the model data for the uk region
model_temp_uk = load_model_data(model_variable=model_var,
                                model=test_model,
                                experiment=experiment,
                                start_year=1960,
                                end_year=2018,
                                avg_period=1,
                                grid=dicts.uk_grid)

In [ ]:
# Peform the bootstrapping for the model data
model_stats_uk = model_stats_bs(model=model_temp_uk.mean(axis=2),
                                nboot=10000,)

In [ ]:
# Load the observed data
obs_data_uk = load_obs_data(obs_variable=obs_var,
                            regrid_obs_path="/gws/nopw/j04/canari/users/benhutch/ERA5/t2m_ERA5_regrid_HadGEM.nc",
                            start_year=1960,
                            end_year=2018,
                            avg_period=1,
                            grid=dicts.uk_grid)

In [ ]:
# Calculate the observed stats
obs_stats_uk = calculate_obs_stats(obs_data=obs_data_uk[0],
                                      start_year=1960,
                                      end_year=2018,
                                      avg_period=1,
                                      grid=dicts.uk_grid)

In [ ]:
# plot the moments of the distribution
plot_moments(model_stats=model_stats_uk,
             obs_stats=obs_stats_uk
             )

All looks pretty good, apart from the kurtosis. Looks like the mean will need to be corrected by around ~0.4 K as well.

In [ ]:
# Plot the distributions
plot_distribution(model_data=model_temp_uk.mean(axis=2),
                  obs_data=obs_data_uk[0],
                  )

### Fidelity tests for temp ###

Fidelity tests mostly look good. The low bias in temperature for the model should be corrected. I'm not sure whether the temp will need to be detrended as well.

In [ ]:
importlib.reload(sys.modules['functions'])

from functions import *

In [ ]:
# Plot the events
plot_events(model_data=model_temp_uk.mean(axis=2),
            obs_data=obs_data_uk[0],
            obs_stats=obs_stats_uk,
            start_year=1960,
            end_year=2018,
            figsize_y=5,
    )

### Trend in temperature ###

Does this positive trend in temperature need to be corrected?

In [ ]:
# Run the function again, but detrend the data
plot_events(model_data=model_temp_uk.mean(axis=2),
            obs_data=obs_data_uk[0],
            obs_stats=obs_stats_uk,
            start_year=1960,
            end_year=2018,
            figsize_y=5,
            do_detrend=True,
    )

Break this down into winter1, summer and winetr2.